# Imports

In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from skimage import io

# Dataset Loader

In [ ]:
class CatsAndDogDataset(Dataset):
  def __init__(self, csv_file, root_dir, transform=None):
    self.annotations = pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.transform = transform

  def __len__(self):
    return len(self.annotations)

  def __getitem__(self, index):
    img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
    image = io.imread(img_path)
    y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

    if self.transform:
      image = self.transform(image)

    return (image, y_label)

# Set device

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters

In [ ]:
in_channel = 3
num_classes = 10
learning_rate = 1e-3
batch_size = 32
num_epochs = 1

# Load Data

In [ ]:
dataset = CatsAndDogDataset(csv_file = 'cats_dogs.csv', root_dir = 'cats_dogs_resized', transform=transforms.ToTensor())
train_set, test_set = torch.utils.data.random_split(dataset, [20000, 5000])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)


# Model

In [ ]:
model = torchvision.models.googlenet(pretrained=True)
model.to(device)

# Loss and Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0
  model.eval

  with torch.no_grad():
    for x, y in loaderL
    x = x.to(device=device)
    y = y.to(device=device)

    scores = model(x)
    _, predictions = scores.max(1)
    num_correct += (predictions==y).sum()
    num_samples += predictions.size(0)

  print(f'Got {num_correct} / {num_samples}with accuracy {float(num_correct)/float(num_samples) * 100}')
  model.train()

# Train Network

In [ ]:
for epoch in range(num_epochs):
  losses = []

  for batch_idx, (data, targets) in enumerate(train_loader):
      data = data.to(device=device)
      targets = targets.to(device=device)

      scores = model(data)
      loss = criterion(scores, targets)

      losses.append(loss.item())

      optimizer.zero_grad()
      loss.backward()

      optimizer.step()

  print(f'Cost at each {epoch} is {sum(losses)/len(losses)}')
print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)

